# Outline: Accessing NCBI Resources on the Command Line for Biologists 

This Jupyter Notebook contains the background and instructions for the hands-on exercises of this workshop. **WILL FINISH POPULATING THIS OUTLINE**

* [Introduction](#Introduction)
* [Objective 1 - Getting Around on the Command Line](#Objective-1)

# Introduction

**Why command line? What is the benefit to you as a biologist?** 
* More direct interface between you and the computer - more streamlined, less tabs and windows open etc. 
* Interacting with remote/cloud computer
* Many bioinformatics tools are only available, or have more full options, in a CLI 
* Reusability of your work - can build up to writing scripts that can automate things, or retrieve different things with small modifications 
* More opportunities in general for batch processing and automation 

**Examples of a few GENERAL tasks that would benefit a biologist**
* Download sequences listed by name in a text file 
* Run the same filtering pipeline on 10 different Fastq files 
* Run that same filtering pipeline AGAIN after realizing you needed to use a higher quality score cutoff 

**Other Notes:**
* Reminder that to do all of this, you need to be comfortable with the command line basics first
* Basics of what the EDirect toolkit and what it is used for - your access point to interconnected NCBI databases - a great example of command line functionality *Not sure if I should even go into EDirect vs. EUtilities and what EDirect is doing in the background (creating structured URL)*

## About Jupyter Notebooks and this workshop  
This workshop uses a Jupyter Notebook, a platform that allows you to run code from individual cells in a web page and display the results of the command.  

In this notebook the language of the code cells is simply the Unix bash shell that you can use to run command-line tools (MagicBLAST, blastn, efetch, etc.) or invoke any standard Unix programs or utilities (ls, grep, head, tail, sort, cut etc.)

In some cases the commands will create files on your working directory on the server. When that happens a new file will appear in the list on the left-hand side of your notebook.  

To run code in a cell you can select the cell and  use the "Run" button at the top of the notebook, or you can use hold the shift key down and press the enter key (shift+enter) to run the cell.

**Example:** Run the following cell. This will create file on your working directory, list the contents of your directory, and list the contents of your file.

In [ ]:
# Creating a file with Unix
echo "This is my file"

The output appears immediately below the cell. Notice that the new file appears in the list on the left-hand side of your browser. Also, the bracketed space to the left of the cell now contains the number 1 \[1]. The number is the number of times you have run cells in the notebook. When the cell is running the brackets will show an asterisk \[*].

**Important tip:** As you go through the notebook, in order to run, some cells require that you have run the cells before them. If you missed a previous cell, you can use the "Run" menu to "Run All Above the Selected Cell"

**If you get lost**: Click the outline icon on the left sidebar (looks like three bullet points) to get an entire interactive outline of the course. 

### Software installed on this notebook
The following bioinformatics tools are installed on this server for use during the workshop, and will need to be installed locally if you want to re-create these analyses on your own computer. 

- **EDirect**: a suite of scripts for accessing NCBI sequence and literature data through the E-Utilities API. [More info](https://www.ncbi.nlm.nih.gov/books/NBK179288/)


## Our Case Studies

**You are a researcher starting in a new lab working on a Disease/Phenotype: Acute Intermittent Porphyria** 

* Download citations about this disease from PubMed 
* Identify most relevant paper(s) to figure out what gene(s) are involved 
* Find out more about this gene by looking at its entry in the Genes database 
* Look at spread of non-human animals that have related Gene sequences - your lab is interested in moving away from a mouse model
* Download relevant sequence data to prepare for BLAST search

# Objective 1 - Set up a workspace using the command line <a class="anchor" id="Objective-1"></a>

**End Goal**: A main project directory with appropriately named sub-directories for different types of data (citations/publications, nucleotide, protein, genomes)

**Concepts covered:**

* "Anatomy" of a Bash command with, and without a single argument
* Basics of hierarchical Linux directory structure (walk through diagram of structure side by side with actual command line navigation - moving "up" and "down") 
* Navigating/investigating this around this structure (pwd, ls, cd) 
* Modifying this directory structure by: Creating subdirectories, renaming a directory, moving an existing file into a directory you created (mkdir, mv, cp) 

## So, what IS the command line? And why do we use it? 

## Running Commands: Program Name and Parameters

**Commands I will demo during this section:**
* `pwd`
*  `ls` vs. `ls -a` - commands that can be run with or without arguments 
*  `echo` (uh oh, this is missing an argument! ) 
* `echo "this is a message"` this is better! 

In [ ]:
ls --help

## Interacting with Unix directory structure

**Need to create custom directory structure diagram labeled with our JupyterHub directory names**

**Commands I will demo:**
* You are Here:  `pwd`
* What's in a (directory) name/path?
* Checking out our surroundings: `ls` for within a directory, compared with `ls .` and `ls ..` 
* Moving around in our surroundings: `cd` command and its arguments, practicing/predicting what you get if you run `ls` after moving to a different directory.
* Creating your own directories to store data: `mkdir`. 

## Interacting with files

**Commands I will demo during this section:**
* `mv` can be used to literally move a file OR rename a file 
* `cp`
* `less`
* `head`

**Now that you are familiar with the command line, run the following command to finish the setup we need for the rest of the workshop**

In [ ]:
source /srv/scratch/.bash_profile 
# echo $NCBI_API_KEY #Don't worry, you won't see anything happen. 

# Objective 2: Use EDirect commands to identify genes associated with AIP and identify suitable animal models for the disease

**Concepts covered:** 

* Basic esearch command (program name, database, query)
* The result structure they return (focusing on number of results) 
* Making queries clearer/results more specific by adding bracketed field names
* Different databases have different fields - how to use "einfo" to find out more about what databases exist and what type of info can be found in each of them  
* Piping data into efetch commands to get different types of data returned - note that different databases have different structures that can be returned 
* The idea that the EDirect toolkit allows for flexible exploration - we could have gotten to our particular Gene ID "answer" in a number of ways 

## Introduction to the EDirect suite

## Using `einfo` to find out about databases

The `einfo` command is very useful for learning about available databases, the searchable fields in each database, and links between the various NCBI databases. When run with the `dbs` (databases) flag, `einfo` produces a list of available databases.  

In [ ]:
einfo -dbs 

Okay, that is a long list of databases! Sometimes, we want to store a list like this in a text file so that we can examine it again without running the command again. We can use the `>` symbol to direct the output of the `einfo` command into a text file called `ncbi_dbs.txt` that lives in our current directory. 

In [ ]:
einfo -dbs > ncbi_dbs.txt

We can go to our files pane (click on folder icon), and double click `ncbi_dbs.txt` to open that list in another tab for future reference! We can look at the list and guess that `pubmed` might be a good option for looking at publications. To check, we can run `einfo` again, this time specifying `pubmed`. 

In [ ]:
einfo -db pubmed

This output (unless you are already fluent in XML) is a little bit hard to read. Lets add the `-fields` flag to get a human-readable list of the searchable fields in `pubmed`, and also direct this list to a file. This will come in handy soon as we refine our searches. Take a look! 

In [ ]:
einfo -db pubmed -fields > pubmed_fields.txt

In [ ]:
# Briefly compare with the fields in a sequence database, then clear the output
einfo -db nuccore -fields

## Using `esearch` and `efetch` to find relevant results in databases. 

As the name might suggest, we have `esearch` allows us to perform an Entrez search of a database using a query. For our purposes, a basic `esearch` command will have a database specified using `-db` (remember to look at our list!) and something that we are searching for inside of that database, specified using `-query`. 

In [ ]:
# Demonstrate that running `esearch` without specifying a value for one or both of these will return errors
esearch -db pubmed
esearch -query "aip" 

First, lets try the full name of the disease as a query. 

In [ ]:
# Looking for a disease of interest to us...  
esearch -db pubmed -query "acute intermittent porphyria"

This results structure tells us a number of useful things, and also stores this information in way that that can feed into downstream programs. First, it reminds us what database we just searched in, and stores this info for use in other programs. We also may want to look at the `Count` value, which counts the number of records returned by the search. 

As March 2023, there are **2353 results**, too many papers for us to read, and probably too many for us to even download. Note also that this the `esearch` step in and of itself does NOT give us any info about the contents of the results. 

### Refining our search using database fields

If we want to get a more manageable number of results, one way to do that is to modify the query with an applicable **field** from the PubMed database. Take a look back at our `pubmed_fields.txt` file to get an idea of their names and abbreviations. **Let's modify our query to look only at papers published in 2020:** 

In [ ]:
esearch -db pubmed -query "acute intermittent porphyria AND 2020 [PDAT]"

This time we see many fewer results! What did we modify in our query? 
* `AND`: This tells **esearch** we are qualifying the first half of the query, which is our disease name, with another condition. In this case, a publication year. 
* `2020`: Our chosen value for specifying publication date 
* `[PDAT]`: This tells **esearch** which PubMed database field `2020` should be searched in for. Note that this term matches exactly what we got from `einfo`, and is placed inside of brackets. 

You can apply this same logic to choosing a specific journal. Let's assume that since we are interested in the genetic basis of AIP, the journal **Genes** might be a good option: 

In [ ]:
esearch -db pubmed -query "acute intermittent porphyria AND Genes [JOUR]"

As of March 2023, this returned just a single result, much more manageable. We can actually get the same result if we specify BOTH a journal and year: 

In [ ]:
esearch -db pubmed -query "acute intermittent porphyria AND Genes [JOUR] AND 2020 [PDAT]"

### Sending `esearch` results to `efetch`

Now that we have narrowed our search down to what looks like just one publication, we can retrieve more information about it. We do so by adding in a second command, `efetch` that can receive that result structure directly from `esearch` and return information about that reuslt in a format of our choice. We can use one short command to see what formats `efetch` can return for us for each database: 

In [ ]:
efetch -help

In [ ]:
esearch -db pubmed -query "acute intermittent porphyria AND Genes [JOUR]" | efetch -format abstract

In [ ]:
# You can also direct this to a file to save it for later 
esearch -db pubmed -query "acute intermittent porphyria AND Genes [JOUR]" | efetch -format abstract > AIP_genes.txt

In [ ]:
head AIP_genes.txt
# mv AIP_genes.txt PubMed/

In [ ]:
#Medline format, ready for your citation manager
esearch -db pubmed -query "acute intermittent porphyria AND Genes [JOUR]" | efetch -format medline

In [ ]:
# And finally, if you just want PubMed ID to pipe into something else
esearch -db pubmed -query "acute intermittent porphyria AND Genes [JOUR]" | efetch -format uilist

In [ ]:
#esearch -db pubmed -query "acute intermittent porphyria AND Genes [JOUR]" | elink -target gene

### Using the NCBI `Gene` Database

By reading over just this one abstract, we learn that this form of porphyria is related to decreased activity of hepatic hydroxymethylbilane synthase (HMBS), the third enzyme in the heme biosynthetic pathway. Now we know what our target gene is, and can search for it in any number of NCBI databases. 

In [ ]:
# esearch -db pubmed -query "acute intermittent porphyria" | elink -target gene 

In [ ]:
# search -db pubmed -query "acute intermittent porphyria AND Genes [JOUR]" | elink -target gene | efetch

In [ ]:
# Seems obvious to see what "Gene" has to say about a gene...
einfo -db gene -fields

In [ ]:
esearch -db gene -query "HMBS" 

In [ ]:
#Specify that HMBS is actually the official gene symbol, not just a set of characters to search for 
esearch -db gene -query "HMBS[sym]"

425 results is a number we could possibly put into a spreadsheet and take a look at, so lets go ahead and do that, using `efetch` and specifying a tabular output format. Tabular output format can be really nice because this can now be imported into your favorite statistics software, such as Excel or R. 

In [ ]:
esearch -db gene -query "HMBS[sym]" | efetch -format tabular > hmbs_tabular.tsv

In [ ]:
# How many results did we get? This should match what we got from esearch, plus a header. The -l flag specifies that we want to count units by line. 
wc -l hmbs_tabular.tsv

So how can we interpret the results in this table? They represent all the results of searching the **Gene** Database using the string **HMBS** as a **Gene Symbol**.  We can explore the tab-delimited file a bit using the JupyterHub file viewer. 

Here is some information we can learn from browsing the table: 
* The range of organisms that have a gene known as HMBS, and their TaxIDs
* Acccession numbers that will help us to find the actual gene sequences
* Other names and aliases for the genes 
* That HMBS is on human chromosome 1, and rat chromosome 8

If we want more information on just that top result, the well-annotated human sequence, we can look specifically for that Gene ID and produce a **gene table** output. 


In [ ]:
esearch -db gene -query "3145[UID]" | efetch -format gene_table > hmbs_gene_table.txt

In [ ]:
We could have also found the same human HMBS sequence by starting with the disease name: 

In [ ]:
esearch -db gene -query "acute intermittent porphyria"

In [ ]:
esearch -db gene -query "acute intermittent porphyria" | efetch -format tabular > aip_genes.tsv

## Searching a sequence database

So, now that we have identified the exact ID and name of a gene, it is easy to download the actual sequence for the gene in a format of our choice. First, we can look back at our list of NCBI databases and see if one of them might be appropriate. Maybe something like **nuccore** or **protein**! We can start by double checking that **nuccore** has our desired output format - a FASTA file. 

In [ ]:
efetch -help

This time, because we have real sequence accession numbers for mRNA sequences in our Gene Table, we can use `efetch` directly using the accession number of the first mRNA variant listed as the `id` parameter: 

In [ ]:
efetch -db nuccore -id XM_005271531.2 -format fasta

# Objective 3: Use `elink` to more exhaustive search for genes and proteins 

**Concepts covered:** 

* elink to move around between and within databases 
* elink "related" to find the other names of genes in the pathway 
* elink with targeted database to get actual sequences for these genes 
* efilter to restrict linked results to a reasonable subset - probably filtering by our target organism (cat) 
* Overview of Genome/Assembly reports and how to pick a relevant assembly 
* Setting up directories for BLAST but the actual analysis is outside the scope of this course (Peter and Wayne might cover?) 

## Finding connected records in another database 

What if we had wanted to ALL of the FASTA-formatted sequences associated with the Gene ID 3145 (Human HMBS), to serve as a query file for a BLAST search against a different organism's genome? The problem is that FASTA files are not direct output from the Genes database itself. 

In [ ]:
esearch -db gene -query "acute intermittent porphyria" | efetch -format fasta # Does not give the expected result 

Luckily, we have a command called `elink` that has two major capabilities: 
* Finding linked records in a different, target, database
* Otherwise connected records in the same database 

We will go through examples of both! First, take a look at the help docs for `elink`: 

In [ ]:
elink -help

You can also see what type of links the **Gene** database has to other NCBI databases: 

In [ ]:
einfo -db gene -links

**So, let's build a query that does the following:** 

1. Queries **Gene** for 3145 as a UID 
2. Passes that result object to `elink`, specifying that we want to look in a different database
3. Specifies the target second database
4. Specifying the format we want to get out of the command
5. Saves the results in a file instead of printing them out to screen

We can accomplish steps 1-3 just by adding a pipe to `elink` before our call to `efetch`: 

In [ ]:
esearch -db gene -query "3145[UID]" | elink -target nuccore

Then, we can add a third step of passing this info to efetch, which will actually retrieve the FASTA formatted file! We definitely want to direct the output to a file for storage...

In [ ]:
esearch -db gene -query "3145[UID]" | elink -target nuccore | efetch -format fasta > nuccore_3145.fasta #This may take a moment

We can quickly double check that this has the contents and format we expect using the **head** command: 

In [ ]:
head -10 nuccore_3145.fasta

Getting back to one of your original research questions about model organism choice - Another use case of linking to another NCBI DB might be to generate a list of species in the with an HMBS gene. We can start with our original query for the HMBS Gene symbol:

In [ ]:
# esearch -db gene -query "HMBS[sym]" | 

# Objective 4: Combine and modify commands from above examples to create re-usable tools

**Learning outcome**: Identify parameters you could modify to make this stuff super relevant for your own research! 

Let's say that you did this analysis a year ago - how can we download sequences only from the past year? Or from a different organism if the focus of your research changes to a new animal model? 

**Concepts covered:** 
* Linux/unix history commands and saving commands in a text file 
* Opening a text file to edit a parameter in a command - just use the within-JupyterHub text editor for now 
* Running a line of code from a text file: you've written your first shell script! 